In [1]:
from LeukemiaClassifier import *
import torch
import numpy as np
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler

In [2]:
def npy_loader(path):
    with open(path, 'r') as f:
        data = np.loadtxt(f)
    return data

In [3]:
dataset = DatasetFolder("./Embeddings/", loader=npy_loader, extensions='.txt')

class_count = {}
for _, label in dataset:
    if label not in class_count:
        class_count[label] = 0
    class_count[label] += 1
weights = [1.0 / class_count[label] for _, label in dataset]

sampler = WeightedRandomSampler(weights, len(weights))

num_train = 1000
num_val = 100

train_dataset, val_dataset, _ = random_split(dataset, [num_train, num_val, len(dataset) - num_train - num_val])

batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [4]:
model = LeukemiaClassifier()
init_weights(model)
loss, tr_acc, vl_acc = train(model, train_loader, val_loader, initial_lr=0.1, epochs=100)
free_cuda(model)

Training for 100 epochs using cuda...
Epoch 1 |	 Loss: 1.76 |	 Training Accuracy: 0.30 |	 Validation Accuracy: 0.27
Epoch 1 |	 Loss: 1.68 |	 Training Accuracy: 0.14 |	 Validation Accuracy: 0.13
Epoch 1 |	 Loss: 1.67 |	 Training Accuracy: 0.30 |	 Validation Accuracy: 0.27
Epoch 1 |	 Loss: 1.67 |	 Training Accuracy: 0.14 |	 Validation Accuracy: 0.13
Epoch 1 |	 Loss: 1.62 |	 Training Accuracy: 0.14 |	 Validation Accuracy: 0.13
Epoch 1 |	 Loss: 1.61 |	 Training Accuracy: 0.28 |	 Validation Accuracy: 0.31
Epoch 1 |	 Loss: 1.59 |	 Training Accuracy: 0.28 |	 Validation Accuracy: 0.31
Epoch 1 |	 Loss: 1.57 |	 Training Accuracy: 0.24 |	 Validation Accuracy: 0.26
Epoch 1 |	 Loss: 1.54 |	 Training Accuracy: 0.24 |	 Validation Accuracy: 0.26
Epoch 1 |	 Loss: 1.52 |	 Training Accuracy: 0.24 |	 Validation Accuracy: 0.26
Epoch 2 |	 Loss: 1.34 |	 Training Accuracy: 0.24 |	 Validation Accuracy: 0.26
Epoch 2 |	 Loss: 1.28 |	 Training Accuracy: 0.24 |	 Validation Accuracy: 0.26
Epoch 2 |	 Loss: 1.28 |	 T

KeyboardInterrupt: 